In [107]:
import datetime
import os
import numpy as np
import re

体素划分[3,5,2]
A  活度矩阵       [3, 5]
CE 发射计数率矩阵  [3, 5]
F  探测效率矩阵    [5, 3]
CE = A * F
A = CE * F^-1

In [108]:
# 文件路径
file_dir = r'C:\Users\X\PycharmProjects\XProject\macScript'
curr_time = str(datetime.date.today())
mk_dir = file_dir + curr_time

# 测量参数
distance = 500 #探测器与钢箱距离
gridNumOfX = 5
gridNumOfY = 4
gridNumOfZ = 3
XLength = 2438
YLength = 1926
ZLength = 1331

xCellLength = XLength / gridNumOfX
xHalfCellLength = XLength / gridNumOfX / 2
yCellLength = YLength / gridNumOfY
yHalfCellLength = YLength / gridNumOfY / 2
zCellLength = ZLength / gridNumOfZ
zHalfCellLength = ZLength / gridNumOfZ / 2

In [109]:
xPos = np.arange(0, XLength, XLength / gridNumOfX).reshape(1, gridNumOfX)
xPos += xCellLength / 2
yPos = np.arange(0, YLength, YLength / gridNumOfY).reshape(1, gridNumOfY)
yPos += yCellLength / 2 + distance
zPos = np.arange(0, ZLength, ZLength / gridNumOfZ).reshape(1, gridNumOfZ)
zPos += zCellLength / 2
# 网格中心坐标
pos = np.zeros((gridNumOfX, gridNumOfY, gridNumOfZ, 3))
for x in range(0, gridNumOfX):
    for y in range(0, gridNumOfY):
        for z in range(0, gridNumOfZ):
            pos[x][y][z][0] = xPos[0][x]
            pos[x][y][z][1] = yPos[0][y]
            pos[x][y][z][2] = zPos[0][z]

In [110]:
def gen_point_list(num=21, x_start=0,x_end=100,y_start=0,y_end=200,z_start=0,z_end=100):
    str_point_list = []
    gps_point_x = np.linspace(x_start, x_end, num).reshape(-1,1)
    gps_point_y = np.linspace(y_start, y_end, num).reshape(-1,1)
    gps_point_z = np.linspace(z_start, z_end, num).reshape(-1,1)
    gps_point = np.concatenate((gps_point_x,gps_point_y,gps_point_z), axis=1)
    for pos in gps_point:
        str_point = str(pos[0]) +' ' +  str(pos[1]) + ' ' + str(pos[2]) +' ' + 'mm\n'
        str_point_list.append(str_point)
    return str_point_list



In [111]:
def gen_dec_list(pos=pos):
    str_detector_x = "/HPGe_simulation/det/setDetectorMoveX "
    str_detector_y = "/HPGe_simulation/det/setDetectorMoveY "
    str_detector_z = "/HPGe_simulation/det/setDetectorMoveZ "
    str_dec_list = []
    dec_pos = pos[:,0,:,:].reshape(15,-1)
    for dpos in dec_pos:
        str_point = str_detector_x + str(dpos[0]) +'\n' +  str_detector_y + str(0) + '\n' + str_detector_z + str(dpos[2]) +'\n'
        str_dec_list.append(str_point)
    return str_dec_list
print(gen_dec_list(pos))

['/HPGe_simulation/det/setDetectorMoveX 243.8\n/HPGe_simulation/det/setDetectorMoveY 0\n/HPGe_simulation/det/setDetectorMoveZ 221.83333333333334\n', '/HPGe_simulation/det/setDetectorMoveX 243.8\n/HPGe_simulation/det/setDetectorMoveY 0\n/HPGe_simulation/det/setDetectorMoveZ 665.5\n', '/HPGe_simulation/det/setDetectorMoveX 243.8\n/HPGe_simulation/det/setDetectorMoveY 0\n/HPGe_simulation/det/setDetectorMoveZ 1109.1666666666667\n', '/HPGe_simulation/det/setDetectorMoveX 731.4000000000001\n/HPGe_simulation/det/setDetectorMoveY 0\n/HPGe_simulation/det/setDetectorMoveZ 221.83333333333334\n', '/HPGe_simulation/det/setDetectorMoveX 731.4000000000001\n/HPGe_simulation/det/setDetectorMoveY 0\n/HPGe_simulation/det/setDetectorMoveZ 665.5\n', '/HPGe_simulation/det/setDetectorMoveX 731.4000000000001\n/HPGe_simulation/det/setDetectorMoveY 0\n/HPGe_simulation/det/setDetectorMoveZ 1109.1666666666667\n', '/HPGe_simulation/det/setDetectorMoveX 1219.0\n/HPGe_simulation/det/setDetectorMoveY 0\n/HPGe_simulat

In [112]:
def gen_point_str(x_pos=0,y_pos=0,z_pos=0):
    str_point = str(x_pos) +' ' +  str(y_pos) + ' ' + str(z_pos) +' ' + 'mm\n'
    return str_point
def gen_point_mac(thread_num=16, run_num=1e7,x_pos=0,y_pos=0,z_pos=0):
    str_verbose_level = """
/control/verbose 0
/run/verbose 0
"""
    str_gps_point = """
/run/numberOfThreads 16
/run/initialize
/gps/particle gamma
/gps/pos/type Point
/gps/pos/centre 1219 981.5 221.8333 mm
/gps/ang/type iso
/gps/ang/mintheta 0 deg
/gps/ang/maxtheta 180 deg
/gps/ang/minphi 0 deg
/gps/ang/maxphi 180 deg
/gps/energy 1.3325 MeV

/run/beamOn 100000000
"""
    gps_pos_patt = re.compile(r"(?<=/gps/pos/centre\s).*")
    threads_patt = re.compile(r"(?<=/run/numberOfThreads\s).*")
    run_patt = re.compile(r"(?<=/run/beamOn\s).*")
    thread_num =  str(thread_num)
    run_num = str(int(run_num))

    str_dec_list = gen_dec_list()
    num = len(str_dec_list)
    mac = [None] * num
    mac_name = [None] * num
    str_point = str(x_pos) +' ' +  str(y_pos) + ' ' + str(z_pos) +' ' + 'mm\n'
    for i, dec_pos in enumerate(str_dec_list):
        mac[i] = str_verbose_level
        mac[i] += dec_pos
        mac[i] += str_gps_point
        mac[i] = re.sub(gps_pos_patt, str_point, mac[i])
        mac[i] = re.sub(threads_patt, thread_num, mac[i])
        mac[i] = re.sub(run_patt, run_num, mac[i])
        mac_name[i] = i
    return mac, mac_name
gen_point_mac(thread_num=16, run_num=1e7,x_pos=0,y_pos=0,z_pos=0)

(['\n/control/verbose 0\n/run/verbose 0\n/HPGe_simulation/det/setDetectorMoveX 243.8\n/HPGe_simulation/det/setDetectorMoveY 0\n/HPGe_simulation/det/setDetectorMoveZ 221.83333333333334\n\n/run/numberOfThreads 16\n/run/initialize\n/gps/particle gamma\n/gps/pos/type Point\n/gps/pos/centre 0 0 0 mm\n\n/gps/ang/type iso\n/gps/ang/mintheta 0 deg\n/gps/ang/maxtheta 180 deg\n/gps/ang/minphi 0 deg\n/gps/ang/maxphi 180 deg\n/gps/energy 1.3325 MeV\n\n/run/beamOn 10000000\n',
  '\n/control/verbose 0\n/run/verbose 0\n/HPGe_simulation/det/setDetectorMoveX 243.8\n/HPGe_simulation/det/setDetectorMoveY 0\n/HPGe_simulation/det/setDetectorMoveZ 665.5\n\n/run/numberOfThreads 16\n/run/initialize\n/gps/particle gamma\n/gps/pos/type Point\n/gps/pos/centre 0 0 0 mm\n\n/gps/ang/type iso\n/gps/ang/mintheta 0 deg\n/gps/ang/maxtheta 180 deg\n/gps/ang/minphi 0 deg\n/gps/ang/maxphi 180 deg\n/gps/energy 1.3325 MeV\n\n/run/beamOn 10000000\n',
  '\n/control/verbose 0\n/run/verbose 0\n/HPGe_simulation/det/setDetectorMo

In [113]:
def output_mac_file(mac_dir=r'..\mac\\'):
    mac, mac_name = gen_point_mac(thread_num=16, run_num=1e7,x_pos=pos[2][0][1][0],y_pos=pos[2][0][1][1],z_pos=pos[2][0][1][2])
    for i,m in enumerate(mac):
        with open(mac_dir+str(mac_name[i]) + '.mac', 'w') as f:
            f.write(m)
            f.close()
print('generate mac succeed')
output_mac_file()

generate mac succeed


In [114]:
new_pos = pos[:,0,:,:].reshape(15,-1)
print(new_pos)
print(new_pos.shape)

[[ 243.8         740.75        221.83333333]
 [ 243.8         740.75        665.5       ]
 [ 243.8         740.75       1109.16666667]
 [ 731.4         740.75        221.83333333]
 [ 731.4         740.75        665.5       ]
 [ 731.4         740.75       1109.16666667]
 [1219.          740.75        221.83333333]
 [1219.          740.75        665.5       ]
 [1219.          740.75       1109.16666667]
 [1706.6         740.75        221.83333333]
 [1706.6         740.75        665.5       ]
 [1706.6         740.75       1109.16666667]
 [2194.2         740.75        221.83333333]
 [2194.2         740.75        665.5       ]
 [2194.2         740.75       1109.16666667]]
(15, 3)
